In [ ]:
__author__ = 'So Negishi'
__copyright__ = 'Copyright 2019, So Negishi'
__license__ = 'GPL'
__version__ = '0.0.1'
__maintainer__ = 'So Negishi'
__email__ = 'sonegishi_2020@depauw.edu'
__status__ = 'Development'

In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import os
import numpy as np
import pandas as pd
from keplergl import KeplerGl
from collections import defaultdict

## 1. Get airport info

In [3]:
airport_cols = ['type', 'name', 'iso_country', 'iso_region', 'iata_code', 'coordinates']
airport_df = pd.read_csv('./raw_data/airport-codes.csv', header=0, usecols=airport_cols)

airport_col_names = ['facility_type', 'name', 'country', 'region', 'iata', 'coordinates']
airport_df.columns = airport_col_names

In [4]:
filter_airports = ['small_airport', 'medium_airport', 'large_airport']
airport_df = airport_df[airport_df.country == 'US']
airport_df = airport_df[airport_df.facility_type.isin(filter_airports)]

coordinates = np.array([tuple(crd.split(',')) for crd in airport_df.coordinates.tolist()])
airport_df['latitude'] = list(map(float, coordinates[:, 0]))
airport_df['longitude'] = list(map(float, coordinates[:, 1]))
airport_df.drop(labels='coordinates', axis='columns', inplace=True)

airport_df.dropna(axis='index', subset=['iata'], inplace=True)
airport_df.reset_index(drop=True, inplace=True)
airport_df

,facility_type,name,country,region,iata,latitude,longitude
0,small_airport,Ocean Reef Club Airport,US,US-FL,OCA,25.325399,-80.274803
1,small_airport,Pilot Station Airport,US,US-AK,PQS,61.934601,-162.899994
2,small_airport,Crested Butte Airpark,US,US-CO,CSE,38.851918,-106.928341
3,small_airport,LBJ Ranch Airport,US,US-TX,JCY,30.251801,-98.622498
4,small_airport,Metropolitan Airport,US,US-MA,PMX,42.223301,-72.311401
...,...,...,...,...,...,...,...
1873,small_airport,Akiachak Airport,US,US-AK,KKI,60.904800,-161.421997
1874,small_airport,Bear Creek 3 Airport,US,US-AK,BCC,63.573316,-156.149454
1875,small_airport,Birch Creek Airport,US,US-AK,KBC,66.274002,-145.824005
1876,small_airport,Copper Center 2 Airport,US,US-AK,CZC,61.941200,-145.294006


In [5]:
airport_df[airport_df.iata == 'JFK']

,facility_type,name,country,region,iata,latitude,longitude
877,large_airport,John F Kennedy International Airport,US,US-NY,JFK,40.639801,-73.7789


In [6]:
airport_map = KeplerGl(height=900, width=800)
airport_map_data = airport_df[['name', 'latitude', 'longitude', 'facility_type']]
airport_map.add_data(data=airport_map_data[airport_map_data.facility_type == 'large_airport'], name='large_airports')
airport_map.add_data(data=airport_map_data[airport_map_data.facility_type == 'medium_airport'], name='medium_airports')
airport_map.add_data(data=airport_map_data[airport_map_data.facility_type == 'small_airport'], name='small_airports')
airport_map.save_to_html(file_name='./visualization/airport_map.html')
# airport_map

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md
Map saved to ./visualization/airport_map.html!


## 2. Get flight info

In [7]:
flight_df = pd.read_csv('./raw_data/266694930_T_ONTIME_REPORTING.csv', header=0)
flight_df.columns = [col.lower() for col in flight_df.columns.tolist()]

/Users/sonegishi/.pyenv/versions/anaconda3-2019.03/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (48,76,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
flight_df

,year,quarter,month,day_of_month,day_of_week,fl_date,op_unique_carrier,op_carrier_airline_id,op_carrier,tail_num,...,div4_tail_num,div5_airport,div5_airport_id,div5_airport_seq_id,div5_wheels_on,div5_total_gtime,div5_longest_gtime,div5_wheels_off,div5_tail_num,unnamed: 109
0,2019,3,9,5,4,2019-09-05,G4,20368,G4,311NV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,3,9,12,4,2019-09-12,G4,20368,G4,241NV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,3,9,16,1,2019-09-16,G4,20368,G4,260NV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019,3,9,20,5,2019-09-20,G4,20368,G4,230NV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019,3,9,27,5,2019-09-27,G4,20368,G4,272NV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605974,2019,3,9,19,4,2019-09-19,WN,19393,WN,N469WN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
605975,2019,3,9,19,4,2019-09-19,WN,19393,WN,N7718B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
605976,2019,3,9,19,4,2019-09-19,WN,19393,WN,N564WN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
605977,2019,3,9,19,4,2019-09-19,WN,19393,WN,N295WN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
flights = flight_df[['origin', 'dest']].to_records(index=False).tolist()     
unique_flights = defaultdict(int)
for key, flight in enumerate(flights):
    unique_flights[flight] += 1

In [10]:
unique_flights_list = list()
for route, num in unique_flights.items():
    unique_flights_list.append((route[0], route[1], num))
unique_flights_list[:10]

[('LAX', 'BLI', 10),
 ('SAV', 'LCK', 9),
 ('JAX', 'ORF', 9),
 ('FWA', 'PIE', 9),
 ('LAS', 'GRR', 18),
 ('FAT', 'LAS', 52),
 ('LEX', 'PIE', 9),
 ('IND', 'LAS', 136),
 ('MDT', 'BNA', 9),
 ('LAS', 'FCA', 9)]

## 3. Export data

### Export edges.csv

In [11]:
edges_df_columns = ['origin', 'dest', 'num_of_flights']
edges_df = pd.DataFrame(unique_flights_list, columns=edges_df_columns)
edges_df.to_csv('./processed_data/edges.csv', index=False)

In [12]:
edges_df

,origin,dest,num_of_flights
0,LAX,BLI,10
1,SAV,LCK,9
2,JAX,ORF,9
3,FWA,PIE,9
4,LAS,GRR,18
...,...,...,...
5498,TVC,ATL,1
5499,AUS,BTR,1
5500,CAK,DCA,30
5501,CRW,DCA,30


### Export vertices.csv

In [13]:
inflow_dict = defaultdict(int)
outflow_dict = defaultdict(int)
for (origin, dest, num) in edges_df.itertuples(index=False):
    inflow_dict[dest] += num
    outflow_dict[origin] += num
outflow_dict

def _get_init_capacity(row):
    curr_airport = row['iata']
    return outflow_dict[curr_airport] + inflow_dict[curr_airport]
airport_df['init_capacity'] = airport_df.apply(_get_init_capacity, axis=1)

In [14]:
def _encode_security_level(row):
    facility_type = row['facility_type']
    if facility_type == 'small_airport':
        return 10
    elif facility_type == 'medium_airport':
        return 20
    elif facility_type == 'large_airport':
        return 30
    else:
        raise
airport_df['security_level'] = airport_df.apply(_encode_security_level, axis=1)

In [15]:
def _get_in_charge(row):
    return True if row['latitude'] < 40 and -130 < row['longitude'] < -85 else False

airport_df['in_charge'] = airport_df.apply(_get_in_charge, axis=1)

In [16]:
airport_df.describe()

,latitude,longitude,init_capacity,security_level
count,1878.000000,1878.000000,1878.000000,1878.000000
mean,41.477703,-104.223105,642.888179,15.276890
std,9.986496,26.655076,3799.158579,6.561297
min,19.721399,-176.645996,0.000000,10.000000
25%,34.760301,-117.974752,0.000000,10.000000
50%,39.755350,-96.756649,0.000000,10.000000
75%,44.479100,-84.787476,0.000000,20.000000
max,71.285402,174.113998,63901.000000,30.000000


In [17]:
airport_df = airport_df[['iata', 'name', 'country', 'region', 'latitude', 'longitude', 'facility_type', 'init_capacity', 'security_level', 'in_charge']]
airport_df.to_csv('./processed_data/vertices.csv', index=False)

In [18]:
airport_df

,iata,name,country,region,latitude,longitude,facility_type,init_capacity,security_level,in_charge
0,OCA,Ocean Reef Club Airport,US,US-FL,25.325399,-80.274803,small_airport,0,10,False
1,PQS,Pilot Station Airport,US,US-AK,61.934601,-162.899994,small_airport,0,10,False
2,CSE,Crested Butte Airpark,US,US-CO,38.851918,-106.928341,small_airport,0,10,True
3,JCY,LBJ Ranch Airport,US,US-TX,30.251801,-98.622498,small_airport,0,10,True
4,PMX,Metropolitan Airport,US,US-MA,42.223301,-72.311401,small_airport,0,10,False
...,...,...,...,...,...,...,...,...,...,...
1873,KKI,Akiachak Airport,US,US-AK,60.904800,-161.421997,small_airport,0,10,False
1874,BCC,Bear Creek 3 Airport,US,US-AK,63.573316,-156.149454,small_airport,0,10,False
1875,KBC,Birch Creek Airport,US,US-AK,66.274002,-145.824005,small_airport,0,10,False
1876,CZC,Copper Center 2 Airport,US,US-AK,61.941200,-145.294006,small_airport,0,10,False


In [19]:
airport_df[airport_df.iata == 'IND']

,iata,name,country,region,latitude,longitude,facility_type,init_capacity,security_level,in_charge
843,IND,Indianapolis International Airport,US,US-IN,39.7173,-86.294403,large_airport,7621,30,True


In [20]:
airport_df[airport_df.in_charge == True].describe()

,latitude,longitude,init_capacity,security_level
count,675.000000,675.000000,675.000000,675.000000
mean,34.931215,-102.017317,714.682963,15.362963
std,3.039682,11.599152,3918.500156,6.780369
min,25.906799,-123.795998,0.000000,10.000000
25%,32.704201,-112.369499,0.000000,10.000000
50%,35.040199,-98.809799,0.000000,10.000000
75%,37.564550,-92.221947,0.000000,20.000000
max,39.943902,-85.027496,51590.000000,30.000000


In [21]:
airport_df.describe()

,latitude,longitude,init_capacity,security_level
count,1878.000000,1878.000000,1878.000000,1878.000000
mean,41.477703,-104.223105,642.888179,15.276890
std,9.986496,26.655076,3799.158579,6.561297
min,19.721399,-176.645996,0.000000,10.000000
25%,34.760301,-117.974752,0.000000,10.000000
50%,39.755350,-96.756649,0.000000,10.000000
75%,44.479100,-84.787476,0.000000,20.000000
max,71.285402,174.113998,63901.000000,30.000000
